In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
basedir = './국토교통부_건축물대장_표제부/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.csv')]
filenames

['bdinfo_seoul_2014.csv',
 'bdinfo_seoul_2015.csv',
 'bdinfo_seoul_2016.csv',
 'bdinfo_seoul_2017.csv',
 'bdinfo_seoul_2018.csv',
 'bdinfo_seoul_2019.csv',
 'bdinfo_seoul_2020.csv']

In [6]:
bdinfo_df = pd.read_csv(basedir + filenames[0])
print(bdinfo_df.shape)
bdinfo_df.head(1)

C:\Users\나\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (14,15,60,61) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(644134, 78)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,내진_설계_적용_여부,내진_능력,시
0,11305-10019310902,2,집합,3.0,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305.0,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1.0,NaN,1.130541e+11,10301.0,0.0,36.0,0.0,삼일교회,0.0,주건축물,NaN,393.0,233.81,59.49,761.29,761.29,193.71,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),04000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3.0,0.0,12.2,4.0,0.0,0.0,0.0,0.0,0.0,761.29,0.0,0.0,0.0,0.0,0.0,0.0,7.0,80.5,20070821,20070829,20101216.0,2007,3080079.0,건축과,1101.0,신축허가,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,20101228.0,서울특별시


In [7]:
bdinfo_df['대장_구분_코드_명'].value_counts()

일반    514634
집합    129500
Name: 대장_구분_코드_명, dtype: int64

In [8]:
bdinfo_df['주_부속_구분_코드'].value_counts()

가동          1960
나동          1899
101동        1781
102동        1473
A동          1222
            ... 
성도아트빌아파트       1
참좋은 빌          1
창대빌라 A동        1
부3             1
동아빌리지          1
Name: 주_부속_구분_코드, Length: 17714, dtype: int64

In [10]:
bdinfo_df.head()

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,내진_설계_적용_여부,내진_능력,시
0,11305-10019310902,2,집합,3.0,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305.0,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1.0,NaN,1.130541e+11,10301.0,0.0,36.0,0.0,삼일교회,0.0,주건축물,NaN,393.00,233.81,59.49,761.29,761.29,193.71,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),04000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3.0,0.0,12.2,4.0,0.0,0.0,0.0,0.0,0.0,761.29,0.0,0.0,0.0,0.0,0.0,0.00,7.0,80.50,20070821,20070829,20101216.0,2007,3080079.0,건축과,1101.0,신축허가,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,20101228.0,서울특별시
1,11440-2163902,2,집합,3.0,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440.0,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0.0,NaN,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,NaN,177.62,105.30,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4.0,0.0,12.0,4.0,1.0,0.0,0.0,0.0,0.0,393.13,0.0,0.0,0.0,0.0,0.0,0.00,4.0,46.00,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,20101230.0,서울특별시
2,11200-2360202,1,일반,2.0,일반건축물,서울특별시 성동구 송정동 73-566번지,서울특별시성동구 송정14길 17,대한예수교장로회비전교회,11200.0,11800.0,0.0,73.0,566.0,NaN,NaN,NaN,0.0,NaN,1.120041e+11,11801.0,0.0,17.0,0.0,NaN,0.0,주건축물,NaN,806.00,376.05,46.66,1185.07,797.98,99.00,21.0,철근콘크리트구조,"철근콘크리트,철골조",06000,종교시설,교회,10.0,(철근)콘크리트,"슬라브,경량철골조",0.0,0.0,13.2,2.0,1.0,0.0,0.0,0.0,0.0,1185.07,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,NaN,NaN,19781205.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,20110131.0,서울특별시
3,11380-2690602,1,일반,2.0,일반건축물,서울특별시 은평구 응암동 594-87번지,서울특별시은평구 응암로 163-13,NaN,11380.0,10700.0,0.0,594.0,87.0,NaN,NaN,NaN,0.0,NaN,1.138030e+11,10702.0,0.0,163.0,13.0,NaN,0.0,주건축물,NaN,0.00,0.00,0.00,246.15,0.00,0.00,11.0,벽돌구조,연와조,01000,단독주택,"단독주택,다가구용 (6가구)",10.0,(철근)콘크리트,슬라브,0.0,6.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,246.15,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,19900403,NaN,19911128.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,20110105.0,서울특별시
4,11110-10017892502,1,일반,2.0,일반건축물,서울특별시 종로구 평창동 365-1번지,서울특별시종로구 평창11길 45,NaN,11110.0,18300.0,0.0,365.0,1.0,NaN,NaN,NaN,0.0,NaN,1.111041e+11,18301.0,0.0,45.0,0.0,NaN,0.0,주건축물,NaN,356.00,104.22,29.28,404.80,194.31,54.58,21.0,철근콘크리트구조,철근콘크리트구조,03000,제1종근린생활시설,제1종근린생활시설,10.0,(철근)콘크리트,(철근)콘크리트,0.0,0.0,7.9,2.0,1.0,0.0,0.0,0.0,0.0,404.80,0.0,0.0,0.0,0.0,1.0,33.29,2.0,40.19,20070605,20070616,20080319.0,2007,3000082.0,건축과,1101.0,신축허가,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,20110101.0,서울특별시


In [15]:
bdinfo_df.head()

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,내진_설계_적용_여부,내진_능력,시
0,11305-10019310902,2,집합,3.0,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305.0,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1.0,NaN,1.130541e+11,10301.0,0.0,36.0,0.0,삼일교회,0.0,주건축물,NaN,393.00,233.81,59.49,761.29,761.29,193.71,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),04000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3.0,0.0,12.2,4.0,0.0,0.0,0.0,0.0,0.0,761.29,0.0,0.0,0.0,0.0,0.0,0.00,7.0,80.50,20070821,20070829,20101216.0,2007,3080079.0,건축과,1101.0,신축허가,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,20101228.0,서울특별시
1,11440-2163902,2,집합,3.0,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440.0,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0.0,NaN,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,NaN,177.62,105.30,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4.0,0.0,12.0,4.0,1.0,0.0,0.0,0.0,0.0,393.13,0.0,0.0,0.0,0.0,0.0,0.00,4.0,46.00,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,20101230.0,서울특별시
2,11200-2360202,1,일반,2.0,일반건축물,서울특별시 성동구 송정동 73-566번지,서울특별시성동구 송정14길 17,대한예수교장로회비전교회,11200.0,11800.0,0.0,73.0,566.0,NaN,NaN,NaN,0.0,NaN,1.120041e+11,11801.0,0.0,17.0,0.0,NaN,0.0,주건축물,NaN,806.00,376.05,46.66,1185.07,797.98,99.00,21.0,철근콘크리트구조,"철근콘크리트,철골조",06000,종교시설,교회,10.0,(철근)콘크리트,"슬라브,경량철골조",0.0,0.0,13.2,2.0,1.0,0.0,0.0,0.0,0.0,1185.07,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,NaN,NaN,19781205.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,20110131.0,서울특별시
3,11380-2690602,1,일반,2.0,일반건축물,서울특별시 은평구 응암동 594-87번지,서울특별시은평구 응암로 163-13,NaN,11380.0,10700.0,0.0,594.0,87.0,NaN,NaN,NaN,0.0,NaN,1.138030e+11,10702.0,0.0,163.0,13.0,NaN,0.0,주건축물,NaN,0.00,0.00,0.00,246.15,0.00,0.00,11.0,벽돌구조,연와조,01000,단독주택,"단독주택,다가구용 (6가구)",10.0,(철근)콘크리트,슬라브,0.0,6.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,246.15,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,19900403,NaN,19911128.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,20110105.0,서울특별시
4,11110-10017892502,1,일반,2.0,일반건축물,서울특별시 종로구 평창동 365-1번지,서울특별시종로구 평창11길 45,NaN,11110.0,18300.0,0.0,365.0,1.0,NaN,NaN,NaN,0.0,NaN,1.111041e+11,18301.0,0.0,45.0,0.0,NaN,0.0,주건축물,NaN,356.00,104.22,29.28,404.80,194.31,54.58,21.0,철근콘크리트구조,철근콘크리트구조,03000,제1종근린생활시설,제1종근린생활시설,10.0,(철근)콘크리트,(철근)콘크리트,0.0,0.0,7.9,2.0,1.0,0.0,0.0,0.0,0.0,404.80,0.0,0.0,0.0,0.0,1.0,33.29,2.0,40.19,20070605,20070616,20080319.0,2007,3000082.0,건축과,1101.0,신축허가,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,20110101.0,서울특별시


In [16]:
bdinfo_df.iloc[:,5]

0          서울특별시 강북구 수유동 484-80번지
1          서울특별시 마포구 서교동 484-16번지
2          서울특별시 성동구 송정동 73-566번지
3          서울특별시 은평구 응암동 594-87번지
4           서울특별시 종로구 평창동 365-1번지
                   ...           
644129    서울특별시 관악구 신림동 1622-12번지
644130     서울특별시 관악구 신림동 624-12번지
644131     서울특별시 중랑구 망우동 514-20번지
644132     서울특별시 관악구 신림동 711-18번지
644133       서울특별시 동대문구 제기동 540번지
Name: 대지_위치, Length: 644134, dtype: object